In [ ]:
import math
import struct
import time
import random
import string
import concurrent.futures

# MD5

In [ ]:
import math
import struct

class Hash_value:
    def __init__(self, hashvalue_hexa):
        self.hashvalue_hexa = hashvalue_hexa
        self.hashvalue_bits = bin(int(hashvalue_hexa, 16))[2:].zfill(len(hashvalue_hexa) * 4)
        self.hashvalue_integer =  int(hashvalue_hexa, 16)
        self.hashvalue_bytes = bytes.fromhex(self.hashvalue_hexa)

class MD5_Hash:
    # Constructor (__init__)
    def __init__(self, type = "regular"):
        # Set all Variables and Constants dependend on wordsize_bit
        if (type == "regular"):
            self.wordsize_bit = 32
            self.bits_length_rep = 'Q' # 64 bit / 8 Byte
            self.padding_length = 64 - 8
        elif (type == "light"):
            self.wordsize_bit = 8
            self.bits_length_rep = 'H' # 16 bit / 2 Byte
            self.padding_length = 16 - 2
        else: raise ValueError("hash type must be either light or regular")
        self.wordsize_byte = int(self.wordsize_bit / 8)
        self.blocksize_byte = self.wordsize_byte * 16
        self.hexa = (1 << (self.wordsize_byte * 8)) - 1
        a0, b0, c0, d0 = 0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476
        self.Ks = [int(abs(math.sin(i + 1)) * (2**self.wordsize_bit)) & self.hexa for i in range(64)]
        self.Ss = list(map(lambda s: s % self.wordsize_bit,[7, 12, 17, 22] * 4 + [5, 9, 14, 20] * 4 + [4, 11, 16, 23] * 4 + [6, 10, 15, 21] * 4))
        self.a0,self.b0,self.c0,self.d0 = map(lambda x: int(hex(x)[2:].upper()[:(self.wordsize_byte * 2)],16), [a0, b0, c0, d0])
        self.hashlength_bit = 4* self.wordsize_bit
        self.hashlength_byte = 4* self.wordsize_byte

    #convert Type
    def convertType(self,m):
        if isinstance(m, bytes):
            return m
        # Make sure m is converted to byte value
        if isinstance(m, str) and all(c in '01' for c in m):  # If input is a bit string
            byte_length = (len(m) + 7) // 8  # Compute required byte length
            return int(m, 2).to_bytes(byte_length, byteorder="big")
        elif isinstance(m, str):  # If input is a message (string)
            return m.encode('utf-8') # bytes
        else:
            raise ValueError("Input must be either a message (string) or a valid bit string.")

    #Padding Function
    def pad(self,m):
        bytes = self.convertType(m)
        original_length_bits = len(bytes) * 8
        bytes += b'\x80' #appends 10000000
        while len(bytes)%(self.blocksize_byte)!=self.padding_length:
            bytes += b'\x00' #appends 00000000
        bytes += struct.pack(f'<{self.bits_length_rep}', original_length_bits) # appends 16 bit /64 bit representation in litte-endian-format of length
        return bytes

    # Rotation function (rotate left)
    def rotate_left(self, x, n):
        # Perform a left rotation on a 32-bit integer 'x' by 'n' positions
        x &= self.hexa
        return ((x << n) | (x >> (self.wordsize_bit - n))) & self.hexa

    #digest
    def digestABCD(self, A,B,C,D):
        raw = sum(value << (self.wordsize_bit * i) for i, value in enumerate([A,B,C,D])).to_bytes(16, byteorder='little')
        hashvalue_hexa = '{:0{width}x}'.format(int.from_bytes(raw[:self.hashlength_byte], byteorder='big'), width = self.wordsize_bit)
        hv = Hash_value(hashvalue_hexa)
        return Hash_value(hashvalue_hexa)

    #Hash
    def processblock(self, block, inits):
        A, B, C, D = inits
        for i in range(64):
            if i <= 15:
                f = (B & C) | (~B & D)
                g = i
            elif i <= 31:
                f = (B & D) | (C & ~D)
                g = (5 * i + 1) % 16
            elif i <= 47:
                f = B ^ C ^ D
                g = (3 * i + 5) % 16
            elif i <= 63:
                f = C ^ (B | ~D)
                g = (7 * i) % 16

            # Calculate the temporary values
            temp = D
            D = C
            C = B
            M = int.from_bytes(block[self.wordsize_byte * g : self.wordsize_byte * g + self.wordsize_byte], byteorder='little')
            B = (B + self.rotate_left(A + f + self.Ks[i] + M, self.Ss[i])) & self.hexa
            A = temp
        return([A,B,C,D])

    def digest(self,m):
        blocks = self.pad(m)
        # Process each block
        A_final,B_final,C_final,D_final = self.a0, self.b0, self.c0, self.d0
        for offset in range(0, len(blocks), self.blocksize_byte):
            block = blocks[ offset : offset + self.blocksize_byte]

            #Compute Hash
            A,B,C,D = self.processblock(block, [A_final,B_final,C_final,D_final])
            A_final, B_final, C_final, D_final = (A_final + A) & self.hexa, (B_final + B) & self.hexa, (C_final + C) & self.hexa, (D_final + D) & self.hexa
        return self.digestABCD(A_final, B_final, C_final, D_final)

#check correctness by comparing with md5 library
import hashlib
bytedata = "Hello World!".encode('utf-8')
md5_hash = hashlib.md5(bytedata).digest()
computed_hash = MD5_Hash().digest(bytedata).hashvalue_bytes
if (md5_hash != computed_hash):
    print(f"Correct hash:  {md5_hash}")
    print(f"Computed hash: {computed_hash}")

def md5_light(input_data: str) -> str:
    # use MD5 - light to calculate hash
    Hash = MD5_Hash('light')
    hashvalue = Hash.digest(input_data)
    return hashvalue.hashvalue_bits

md5_light("hello")

'00011011010100110111110010011110'

# Birthday Attack

In [ ]:
import hashlib
import random
import string
import math
import time

def random_string(length=8):
    """Generate a random string of fixed length."""
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

def md5_hash(data, hashtype):
    """Compute the MD5 hash of the input data."""
    return MD5_Hash(hashtype).digest(data).hashvalue_hexa

def birthday_attack(collision_percentage=50, hashtype = 'regular' , time_limit=2*60*60):
    """
    Performs a Birthday Attack on MD5, allowing control over collision percentage.

    :param collision_percentage: Percentage of the hash (out of 128 bits) that must match.
                                 Can be 25, 50, 75, or 100.
    :param time_limit: Maximum runtime in seconds (default is 2 hours).
    """
    # Validate input
    if collision_percentage not in [25, 50, 75, 100]:
        raise ValueError("collision_percentage must be one of [25, 50, 75, 100]")

    hash_table = {}  # Dictionary to store hashes
    hashsize = 32 if hashtype == 'regular' else 8
    hex_length = math.ceil((collision_percentage / 100) *  hashsize)  # MD5 has 32 hex characters (128 bits)

    print(f"Searching for {collision_percentage}% collision ({hex_length} hex characters)...")

    start_time = time.time()  # Start time tracking

    while True:
        # Check time limit
        elapsed_time = time.time() - start_time
        if elapsed_time >= time_limit:
            print(f"\nTime limit reached ({time_limit/3600} hours). No collision found.")
            return

        msg = random_string()  # Generate a random message
        hash_value = md5_hash(msg,hashtype)[:hex_length]  # Take only the required portion of the hash

        if hash_value in hash_table and hash_table[hash_value] != msg:
            end_time = time.time()  # Stop time tracking
            runtime = end_time - start_time  # Compute total runtime

            print("\nCollision found!")
            print(f"Collision Percentage: {collision_percentage}% ({hex_length * 4} bits)")
            print(f"Message 1: {hash_table[hash_value]} -> {md5_hash(hash_table[hash_value],hashtype)}")
            print(f"Message 2: {msg} -> {md5_hash(msg,hashtype)}")
            print(f"Time taken: {runtime:.2f} seconds ({runtime/60:.2f} minutes)")

            break  # Stop the attack

        hash_table[hash_value] = msg  # Store in dictionary

# Example Usage
print("Collisions for MD5-Light: ")
print("--------------------------------------------------------------")
for collision_percentage in [25, 50, 75, 100]:
  birthday_attack(collision_percentage, 'light')  # Runs for max 2 hours
  print("--------------------------------------------------------------")
print("Collisions for MD5: ")
print("--------------------------------------------------------------")
for collision_percentage in [25, 50, 75, 100]:
  birthday_attack(collision_percentage)  # Runs for max 2 hours
  print("--------------------------------------------------------------")


Collisions for MD5-Light: 
--------------------------------------------------------------
Searching for 25% collision (2 hex characters)...

Collision found!
Collision Percentage: 25% (8 bits)
Message 1: rqyom1kn -> f3af20a2
Message 2: RF6uEyKY -> f32745c5
Time taken: 0.00 seconds (0.00 minutes)
--------------------------------------------------------------
Searching for 50% collision (4 hex characters)...

Collision found!
Collision Percentage: 50% (16 bits)
Message 1: mW4s8XpL -> 5f8a649c
Message 2: Op1hxd5h -> 5f8a4e57
Time taken: 0.04 seconds (0.00 minutes)
--------------------------------------------------------------
Searching for 75% collision (6 hex characters)...

Collision found!
Collision Percentage: 75% (24 bits)
Message 1: aZjuvB36 -> d7dc2881
Message 2: EeuIAQn1 -> d7dc2861
Time taken: 0.44 seconds (0.01 minutes)
--------------------------------------------------------------
Searching for 100% collision (8 hex characters)...

Collision found!
Collision Percentage: 100% (3